In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce/Ecommerce_Return_dataset.csv


In [2]:
import pandas as pd

# Load dataset

In [3]:
df = pd.read_csv("/kaggle/input/ecommerce/Ecommerce_Return_dataset.csv")

# Convert dates

In [4]:
df['Order_Date'] = pd.to_datetime(df['Order_Date'], errors='coerce')

# Handle missing values

In [5]:
df['Return_Reason'] = df['Return_Reason'].fillna("Not Returned")
df['Return_Status'] = df['Return_Status'].fillna("Not Applicable")

# Check for and drop duplicates

In [6]:
df.drop_duplicates(inplace=True)

# Return % by Category

In [7]:
category_return_rate = df.groupby("Category")['Returned'].mean().reset_index()

# Return % by Geography

In [8]:
geo_return_rate = df.groupby("Geography")['Returned'].mean().reset_index()

**Predict Return Probability**# 

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

# Encode categorical variables

In [10]:
X = df[['Category', 'Geography', 'Marketing_Channel', 'Quantity', 'Price_per_Unit', 'Delivery_Time_Days']]
X = pd.get_dummies(X, drop_first=True)


# Target variable

In [11]:
y = df['Returned']

# Split data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model

In [13]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

# Predict and evaluate

In [14]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.50      0.25      0.33         2
weighted avg       1.00      0.50      0.67         2



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Add predicted risk score back to main dataframe

In [15]:
df['Return_Risk_Score'] = model.predict_proba(X)[:, 1]

# Save to Excel for Power BI use

In [16]:
df.to_excel("Ecommerce_Returns_With_Scores.xlsx", index=False)

# Export high-risk products

In [17]:
high_risk = df[df['Return_Risk_Score'] > 0.7]
high_risk.to_csv("High_Risk_Return_Products.csv", index=False)